In [9]:
from __future__ import print_function, division

%matplotlib inline

import numpy as np
import pandas as pd

import random

import thinkstats2
import thinkplot
import first
import statsmodels.formula.api as smf
import patsy
import nsfg

#Exercise 11-1 Try to predict the date of birth using known variables for co-worker. Limit yourself to what is likely known by other people in the pool.
#Create dataframe from all live births regardless of firstborn or other. Pregnancy length must be greater than 30 weeks
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]
#Implementing the variables I think makes the most difference. A big part of what the text has looked at is whether first borns are late, which so far they seem to be either on time (about 29 weeks).
model = smf.ols('prglngth ~ birthord==1 + nbrnaliv>1', data=live)
results = model.fit()
results.summary()

#r-squared is 0.010 and when r-squared is closer to 1 it means the model can explain the variability fo the response data around its mean. If this was higher it would be a better fit.
#Overall this variable is not a good indicator of birth date.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     45.89
Date:                Sun, 09 Feb 2020   Prob (F-statistic):           1.49e-20
Time:                        15:06:52   Log-Likelihood:                -18253.
No. Observations:                8884   AIC:                         3.651e+04
Df Residuals:                    8881   BIC:                         3.653e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.8492      0.028   1386.425      0.000      38.794      38.904
birthord == 1[T.True]     0.1069      0.040      2.662      0.008       0.028       0.186
nbrnaliv > 1[T.True]     -1.4922      0.165     -9.067      0.000      -1.815      -1.170
==============================================================================
Omnibus:                     1575.089   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6117.494
Skew:                          -0.845   Prob(JB):                         0.00
Kurtosis:                       6.698   Cond. No.                         9.32
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
#Exericse 11-3. Try to predict the number of children for a 35 year old, black college graduate whose annual household income exceeds $75,000.
#Define variables. Live is live pregnancies over 30 weeks. Resp is the responses. Join case id will join respondents with the other data. Pulled this info from chap11ex
live = live[live.prglngth>30]
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')
join.numbabes.replace([97], np.nan, inplace=True)
join['age2'] = join.age_r**2
#Using Poisson regression since dependent variable is an integer count (pg 137). Using code from chap11soln
formula='numbabes ~ age_r + age2 + age3 + C(race) + totincr + educat'
formula='numbabes ~ age_r + age2 + C(race) + totincr + educat'
model = smf.poisson(formula, data=join)
results = model.fit()
results.summary() 
#Pulling the columns we want, creating new df with those columns, and then predicting based on those columns. Using code from chap11soln
columns = ['age_r', 'age2', 'age3', 'race', 'totincr', 'educat']
new = pd.DataFrame([[35, 35**2, 35**3, 1, 14, 16]], columns=columns)
results.predict(new)

#Results show this lady is most likely to have 2.5 kids (not possible). Not sure in this case if you would round down or up?

Optimization terminated successfully.
         Current function value: 1.677002
         Iterations 7


0    2.496802
dtype: float64

In [13]:
#Excercise 11-4 Trying to find the likelihood of whether a 25 year old, white, high school graduate with an annual household income of $45,000 is most likely to be married, etc.
#Setting variables. Formula looks at the criteria we need. Model uses mnlogit to use multinomial logistic regression. Then we show the results.
formula='rmarital ~ age_r + age2 + C(race) + totincr + educat'
model = smf.mnlogit(formula, data=join)
results = model.fit()
results.summary() 
#Taking the columns that are necessary for prediction and creating new dataframe. Then using that dataframe to predict the likelihood of cohabitation of this particular woman.
columns = ['age_r', 'age2', 'race', 'totincr', 'educat']
new = pd.DataFrame([[25, 25**2, 2, 11, 12]], columns=columns)
results.predict(new)

#This woman is about 75% likely to be married.

Optimization terminated successfully.
         Current function value: 1.084053
         Iterations 8


,0,1,2,3,4,5
0,0.750028,0.126397,0.001564,0.033403,0.021485,0.067122
